### Разметка твитов Илона Маска

Ссылки

https://www.kaggle.com/code/buyuknacar/comparing-sentiment-classifiers-txblob-vader-bert

https://www.kaggle.com/discussions/questions-and-answers/206813

https://huggingface.co/search/full-text?q=sentiment+twitter+analysis+english&type=model

some models

https://huggingface.co/agentlans/deberta-v3-xsmall-tweet-sentiment

https://huggingface.co/Manas2708/bert-base-sentiment-analysis

https://huggingface.co/siebert/sentiment-roberta-large-english

https://huggingface.co/j-hartmann/emotion-english-distilroberta-base

https://huggingface.co/michellejieli/emotion_text_classifier

https://huggingface.co/Linna/emotion-english-distilroberta-melinna

https://huggingface.co/mrhacker7599/emotion-english-distilroberta-base

https://huggingface.co/MicahB/emotion_text_classifier

https://huggingface.co/pysentimiento/robertuito-sentiment-analysis

https://huggingface.co/pysentimiento/bertweet-pt-sentiment

https://huggingface.co/pysentimiento/robertuito-emotion-analysis

In [3]:
!pip install textblob


   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
    --------------------------------------- 10.2/626.3 kB ? eta -:--:--
    --------------------------------------- 10.2/626.3 kB ? eta -:--:--
   - ------------------------------------- 30.7/626.3 kB 325.1 kB/s eta 0:00:02
   ----- --------------------------------- 92.2/626.3 kB 581.0 kB/s eta 0:00:01
   -------- ----------------------------- 143.4/626.3 kB 708.1 kB/s eta 0:00:01
   ------------ ------------------------- 204.8/626.3 kB 831.5 kB/s eta 0:00:01
   ---------------------- ----------------- 348.2/626.3 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------  624.6/626.3 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 626.3/626.3 kB 1.8 MB/s eta 0:00:00


In [4]:
!pip install vaderSentiment
!pip install transformers
!pip install torch

   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/126.0 kB ? eta -:--:--
   ------ -------------------------------- 20.5/126.0 kB 217.9 kB/s eta 0:00:01
   ------------ -------------------------- 41.0/126.0 kB 326.8 kB/s eta 0:00:01
   ------------------- ------------------- 61.4/126.0 kB 409.6 kB/s eta 0:00:01
   ---------------------- ---------------- 71.7/126.0 kB 391.3 kB/s eta 0:00:01
   -------------------------------------- 126.0/126.0 kB 528.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/43.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.5 kB ? eta -:--:--
     -------------------------- ----------- 30.7/43.5 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 43.5/43.5 kB 266.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB 991.0 kB/s eta 0:00:11
   

In [5]:
import pandas as pd
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re

In [6]:
from textblob.sentiments import NaiveBayesAnalyzer

In [14]:
df = pd.read_csv('elon_musk_tweets.csv')

In [18]:
vader_analyzer = SentimentIntensityAnalyzer()
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

pytorch_model.bin:  80%|#######9  | 535M/669M [00:00<?, ?B/s]

In [20]:
def vader_label(text):
    scores = vader_analyzer.polarity_scores(text)
    return 1 if scores['pos'] > scores['neg'] else 0


In [22]:
def textblob_label(text):
    polarity = TextBlob(text).sentiment.polarity
    return 1 if polarity >= 0 else 0

In [24]:
def bert_label(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
  
    sentiment_class = torch.argmax(probs).item()
    return 1 if sentiment_class >= 3 else 0  

In [30]:
df['vader_feeling'] = df['text'].apply(vader_label)

In [32]:
df['textblob_feeling'] = df['text'].apply(textblob_label)

In [34]:
df['bert_feeling'] = df['text'].apply(bert_label)

In [35]:
def calculate_accuracy(true_labels, predicted_labels):
    return (true_labels == predicted_labels).mean()

In [36]:
vader_accuracy = calculate_accuracy(df['feeling'], df['vader_feeling'])
textblob_accuracy = calculate_accuracy(df['feeling'], df['textblob_feeling'])
bert_accuracy = calculate_accuracy(df['feeling'], df['bert_feeling'])

KeyError: 'feeling'

In [39]:
print(f"Vader Accuracy: {vader_accuracy:.2f}")
print(f"TextBlob Accuracy: {textblob_accuracy:.2f}")
print(f"BERT Accuracy: {bert_accuracy:.2f}")

NameError: name 'vader_accuracy' is not defined